# Compute the NLL metric before and after tracking

In [2]:
import numpy as np
import torch
import math
import os

In [12]:
before_path = "../det/check/check_loss_two_step_center_rh_ind/"
after_path = "./byte_tracker/cov/"
after_cfk_path = after_path + "cfk/"
after_nlla_path = after_path + "nlla/"
after_origin_path = after_path + "origin/"
after_cfk_nlla_path = after_path + "cfk_nlla/"
gt_path = "./TrackEval/data/gt/mot_challenge/"
scene_idxes_file_path = "../utils/test_scenes.txt"
var_cp_dict = {"upperbound": [156.3751341925818, 65.24260517866209, 2785.7870238883734, 26.43169171689354], "disco": [214.13191750615232, 77.15047953920649, 2503.1216487206066, 24.387592952924123], "lowerbound": [268.0828005828979, 93.20827260609053, 18277.14814591181, 40.319136673153885]}

In [13]:
def convert_bbox_to_z(bbox):
    """
    Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
      [x,y,r,h] where x,y is the centre of the box and r is
      the aspect ratio
    """
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = bbox[0] + w / 2.0
    y = bbox[1] + h / 2.0
    if float(h) < 0.00001:
        print([x, y, w, h])
        print(bbox)
    r = w / float(h)
    return np.array([x, y, r, h])

def split_data_by_frame(data):
    res = []
    start = 0
    for i in range(1,data.shape[0]):
        if data[i][0] != data[start][0]:
            res.append(data[start:i,:])
            start = i
    res.append(data[start:,:])
    return res

def iou_batch(bb_test, bb_gt):
    """
    From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
    """
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_test = np.expand_dims(bb_test, 1)

    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0.0, xx2 - xx1)
    h = np.maximum(0.0, yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])
        + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1])
        - wh
    )
    return o

def linear_assignment(cost_matrix):
    try:
        import lap

        _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
        return np.array([[y[i], i] for i in x if i >= 0])  #
    except ImportError:
        from scipy.optimize import linear_sum_assignment

        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))

def pre_process_bbox(datas):
    """
    make the bbox be [x1,y1,x2,y2] where [x1,y1] is the left bottom and [x2,y2] is the right top
    """
    for i in range(datas.shape[0]):
        small = [min(datas[i][0], datas[i][2]), min(datas[i][1], datas[i][3])]
        large = [max(datas[i][0], datas[i][2]), max(datas[i][1], datas[i][3])]
        datas[i][0] = small[0]
        datas[i][1] = small[1]
        datas[i][2] = large[0]
        datas[i][3] = large[1]
    return datas

def remove_error_bbox(dets):
    res = []
    for i in range(dets.shape[0]):
        if dets[i, 0] == dets[i, 2] or dets[i, 1] == dets[i, 3]:
            continue
        res.append(list(dets[i]))
    return np.array(res)

def convert_bboxs_to_zs(datas):
    for i in range(datas.shape[0]):
        datas[i, :4] = convert_bbox_to_z(datas[i,:4])
    return datas
    
def compute_nll(dets, gts, mode, is_before_tracking, iou_threshold, is_use_cp_cov):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = torch.from_numpy(convert_bboxs_to_zs(matched_dets[:,:4]))[:,:2]
    target = torch.from_numpy(convert_bboxs_to_zs(matched_gts[:,:4]))[:,:2]
    if is_before_tracking:
        cov = np.exp(matched_dets[:,8:10])
    else:
        cov = matched_dets[:,8:10]
    if is_use_cp_cov:
        cp_cov = np.array(var_cp_dict[mode])[:2]
        cov = cov * cp_cov
    cov = torch.from_numpy(cov)
    std = torch.sqrt(cov)
    predicted_normal_dists = torch.distributions.normal.Normal(pred, std)
    negative_log_prob = - predicted_normal_dists.log_prob(target)
    negative_log_prob = torch.clamp(negative_log_prob, min = 0)
    negative_log_prob = torch.sum(negative_log_prob,axis=1)/4
    # if sum(negative_log_prob) > 10*7:
    #     print(dets)
    #     print(gts)
    #     print(matched_indices)
    #     print(negative_log_prob)
    #     print(iou_matrix)
    #     print(matched_dets)
    #     print(matched_gts)
    #     return [], []
    return negative_log_prob.tolist(), matched_indices

def compute_nll_for_mode(mode, det_path, is_before_tracking, iou_threshold, is_use_cp_cov):
    upper_path = det_path + mode + "/no_rsu/"
    scene_idxes_file = open(scene_idxes_file_path, "r")
    scene_idxes = [int(line.strip()) for line in scene_idxes_file]
    #print(scene_idxes)
    nll_res = []
    for agent in range(1,5):
        det_files_path = upper_path + "tracking" + str(agent)
        gt_files_path = gt_path + "V2X-test" + str(agent)
        for scene in scene_idxes:
            scene = 29
            if is_before_tracking:
                det_scene_file = os.path.join(det_files_path, "det_" + str(scene) + ".txt")
            else:
                det_scene_file = os.path.join(det_files_path, str(scene) + ".txt")
            gt_scene_file = os.path.join(gt_files_path, str(scene), "gt/gt.txt")
            det_datas = np.loadtxt(det_scene_file, delimiter=",")
            gt_datas = np.loadtxt(gt_scene_file, delimiter=",")
            if len(gt_datas) == 0 or len(det_datas) == 0:
                continue
            det_datas = split_data_by_frame(det_datas)
            gt_datas = split_data_by_frame(gt_datas)
            gt_idx = 0
            for det_idx in range(len(det_datas)):
                if det_datas[det_idx][0][0] != gt_datas[gt_idx][0][0]:
                    print(f"Error idx not match {det_idx}, {gt_idx}")
                    continue
                if gt_idx >= len(gt_datas):
                    print(f"gt_idx out of range, {gt_idx} {det_idx}")
                    break
                nll, matched_indices = compute_nll(det_datas[det_idx], gt_datas[gt_idx], mode, is_before_tracking, iou_threshold, is_use_cp_cov)
                # if nll == []:
                #     print(det_scene_file)
                #     print(det_datas[det_idx])
                #     return 0
                nll_res.extend(nll)
                gt_idx += 1
    nll_mean = sum(nll_res) / len(nll_res)
    return nll_mean

def show_one_result(path, is_before_tracking=True, iou_threshold = 0.5, is_use_cp_cov=True):
    mode_list = ["disco", "upperbound","lowerbound"]
    for mode in mode_list:
        nll = compute_nll_for_mode(mode, path, is_before_tracking, iou_threshold, is_use_cp_cov)
        print(f"NLL for {mode} is {nll}")

## before tracking

In [8]:
mode = "upperbound"
nll = compute_nll_for_mode(mode, before_path, True, 0.5, True)
print(f"NLL for {mode} is {nll}")

NLL for upperbound is 27.052949775188047


In [9]:
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for disco is 30.855707534669378
NLL for upperbound is 27.052949775188047
NLL for lowerbound is 22.29408090791383


In [10]:
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 4039.0093437530622
NLL for upperbound is 2002.959383127136
NLL for lowerbound is 2778.6865200689817


## after tracking origin

In [14]:
show_one_result(after_origin_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 1.454157269122955
NLL for upperbound is 1.378421960299013
NLL for lowerbound is 1.8294687387450792


In [18]:
show_one_result(after_origin_path, False, iou_threshold=0.1, is_use_cp_cov=False)

NLL for disco is 14.072119425209033
NLL for upperbound is 1.5882773976391065
NLL for lowerbound is 5.4951381633231735


## after tracking cfk

In [15]:
show_one_result(after_cfk_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 13.356071145606775
NLL for upperbound is 12.823762957483254
NLL for lowerbound is 11.009024149412067


## after tracking nlla

In [16]:
show_one_result(after_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 1.689413071569346
NLL for upperbound is 1.397555385512109
NLL for lowerbound is 3.036706364190975


## after tracking cfk + nlla

In [17]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 15.298357884886856
NLL for upperbound is 13.096550476554423
NLL for lowerbound is 17.160025917575766


In [19]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.1, is_use_cp_cov=False)

NLL for disco is 125.55790246147411
NLL for upperbound is 14.14936376558908
NLL for lowerbound is 27.493608669794956
